In [1]:
import tensorflow as tf

epochs = 50
batch_size = 200
steps_per_epoch = 5000//batch_size
summary = True

In [2]:
def mnist_reader(part):
    for i in range(10):
        with open('Part3_{}_{}.csv'.format(i, part)) as f:
            for line in f:
                yield line.strip(), i

def mnist_decoder(csv_line, label):
    FIELD_DEFAULTS = [[0.0]]*(28*28)
    with tf.variable_scope('DataSource'):
        fields = tf.decode_csv(csv_line, FIELD_DEFAULTS)
        im = tf.stack(fields)
        im = tf.reshape(im, (28, 28, 1))
        return im, tf.one_hot(label, depth=10)

In [3]:
tf.reset_default_graph()

with tf.variable_scope('DataSource'):
    dataset = tf.data.Dataset.from_generator(lambda: mnist_reader('Train'),
        # csv_line, label
        (tf.string, tf.int32),
        (tf.TensorShape([]), tf.TensorShape([])
    )).map(mnist_decoder, num_parallel_calls=2) \
      .batch(batch_size) \
      .prefetch(1) \
      .repeat(epochs)
    
    dataset_val = tf.data.Dataset.from_generator(lambda: mnist_reader('Test'),
        (tf.string, tf.int32),
        (tf.TensorShape([]), tf.TensorShape([])
    )).map(mnist_decoder, num_parallel_calls=2) \
      .batch(batch_size) \
      .prefetch(1)

    iter_handle = tf.placeholder(tf.string, shape=[])
    data_iterator = tf.data.Iterator.from_string_handle(iter_handle, dataset.output_types, dataset.output_shapes)
    train_iterator = dataset.make_one_shot_iterator()
    val_iterator = dataset_val.make_initializable_iterator()
    val_init_op = data_iterator.make_initializer(dataset_val)
    images, onehot_labels = data_iterator.get_next()

with tf.variable_scope('CNN'):
    convmaps = tf.keras.layers.Conv2D(16, (7,7), activation='tanh')(images)
    features = tf.reshape(convmaps, (batch_size, 16*22*22))
    fc1 = tf.keras.layers.Dense(128, activation='tanh')(features)
    pred = tf.keras.layers.Dense(10, activation='sigmoid')(features)
    
with tf.variable_scope('LossFn'):
    total_loss = tf.reduce_sum(tf.square(onehot_labels-pred), axis=1)

with tf.variable_scope('Optimizer'):
    global_step = tf.Variable(0)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    train_op = optimizer.minimize(total_loss, global_step)
    
with tf.variable_scope('Metrics'):
    labels = tf.to_int32(tf.argmax(onehot_labels, axis=1))
    argmax_idx = tf.to_int32(tf.argmax(pred, axis=1))
    epoch_loss_avg, epoch_loss_avg_update = tf.metrics.mean(total_loss)
    epoch_accuracy, epoch_accuracy_update = tf.metrics.accuracy(labels, argmax_idx)
    if summary:
        summary_loss = tf.summary.scalar("loss", epoch_loss_avg)
        summary_vloss = tf.summary.scalar("vloss", epoch_loss_avg)
        summary_acc = tf.summary.scalar("acc", epoch_accuracy)
        summary_vacc = tf.summary.scalar("vacc", epoch_accuracy)
        summary_train = tf.summary.merge([summary_loss, summary_acc])
        summary_val = tf.summary.merge([summary_vloss, summary_vacc])
    
with tf.variable_scope('Initializer'):
    init_global = tf.global_variables_initializer()
    init_local = tf.local_variables_initializer()

In [4]:
from ipywidgets import IntProgress
from IPython.display import display

progress_epoch = IntProgress(description='epoch', min=0, max=epochs-1)
progress_step = IntProgress(description='step', min=0, max=5000//batch_size-1)
progress_valstep = IntProgress(description='val_step', min=0, max=5000//batch_size-1)
display(progress_epoch)
display(progress_step)
display(progress_valstep)

config = tf.ConfigProto(
    intra_op_parallelism_threads=2,
    inter_op_parallelism_threads=2,
    allow_soft_placement=True)

with tf.Session(config = config) as sess:
    sess.run(init_global)
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    for epoch in range(epochs):
        progress_epoch.value = epoch
        sess.run(init_local)
        for step in range(steps_per_epoch):
            progress_step.value = step
            sess.run([train_op,
                      total_loss,
                      epoch_loss_avg_update,
                      epoch_accuracy_update],
                     feed_dict={iter_handle: train_handle})
            
        sess.run([init_local, val_init_op], feed_dict={iter_handle: val_handle})
        for val_step in range(steps_per_epoch):
            progress_valstep.value = val_step
            sess.run([total_loss,
                      epoch_loss_avg_update,
                      epoch_accuracy_update],
                     feed_dict={iter_handle: val_handle})
        print(sess.run(epoch_accuracy))

IntProgress(value=0, description='epoch', max=49)

IntProgress(value=0, description='step', max=24)

IntProgress(value=0, description='val_step', max=24)

0.1488
0.361
0.4628
0.5076
0.5412
0.57
0.6104
0.6624
0.7022
0.7292
0.7464
0.7586
0.7666
0.7768
0.7848
0.7924
0.8
0.8064
0.8106
0.8156
0.8184
0.8224
0.8272
0.8318
0.8364
0.8396
0.843
0.8446
0.8472
0.8486
0.8506
0.8518
0.8528
0.8548
0.8558
0.856
0.8578
0.8582
0.859
0.8602
0.8608
0.8618
0.8628
0.8632
0.8638
0.8654
0.8656
0.866
0.8658
0.8662
